# Region Normalizer

Этот ноутбук демонстрирует, как использовать инструмент нормализации наименований российских регионов. Он помогает легко стандартизировать различные формы наименований регионов России в соответствии с эталонным справочником ([regions_etalon_v2.0.yaml](https://github.com/tochno-st/reg_normalizer/blob/main/data/interim/regions_etalon_v2.0.yaml)).

## 0. Установка

In [8]:
from reg_normalizer import RegionMatcher
import pandas as pd

## 1. Базовое использование — нормализация названия одного региона

In [3]:
matcher = RegionMatcher()

# Пример 1: Сокращение в названии региона

region_name = "московск область"
match, score = matcher.find_best_match(region_name)
print(f"Input: {region_name}")
print(f"Match: {match}")
print(f"Score: {score:.2f}")
print("-" * 50)

Input: московск область
Match: Московская область
Score: 97.00
--------------------------------------------------


In [4]:
# Пример 2: Использование аббревиатуры региона

region_name = "мо"
match, score = matcher.find_best_match(region_name)
print(f"Input: {region_name}")
print(f"Match: {match}")
print(f"Score: {score:.2f}")
print("-" * 50)

Input: мо
Match: Московская область
Score: 100.00
--------------------------------------------------


In [5]:
# Пример 3: Латинские буквы, смешанные с кириллицей
region_name = "Mосковская област"  # Латинская 'M' + опечатка
match, score = matcher.find_best_match(region_name)
print(f"Input: {region_name}")
print(f"Match: {match}")
print(f"Score: {score:.2f}")
print("-" * 50)

Input: Mосковская област
Match: Московская область
Score: 98.50
--------------------------------------------------


## 2. Продвинутое сопоставление с пользовательскими параметрами

In [6]:
custom_weights = {
    'levenshtein': 0.3,  # Меньший вес на точное совпадение символов
    'token_set': 0.7    # Больший вес на совпадение по словам
}

custom_approach_weights = {
    'original': 0.2,     # Меньший вес на оригинальный текст
    'stemmed': 0.8       # Больший вес на стеммированный текст
}

region_name = "свердловск"  # Пропущено 'область'
match, score = matcher.find_best_match(
    region_name,
    weights=custom_weights,
    approach_weights=custom_approach_weights,
    threshold=60  # Пониженный порог
)

print(f"Input: {region_name}")
print(f"Match: {match}")
print(f"Score: {score:.2f}")
print("-" * 50)

Input: свердловск
Match: Свердловская область
Score: 87.16
--------------------------------------------------


## 3. Пакетная обработка с использованием DataFrame

In [9]:
sample_data = pd.DataFrame({
    'region_name': [
        'московск Обл',           # Сокращённая форма
        'свердловск',             # Без слова "область"
        'петербург',              # Сокращённо
        'Mосковская област',      # Латинская 'M' + опечатка
        'татарстан респ.',        # Аббревиатура
        'Свердлов обл',           # Другое окончание
        'aлтайский к',            # Латинская 'a' + аббревиатура
        'Республика     Алтай',   # Лишние пробелы
        'ХМао',                   # Смешанный регистр аббревиатуры
        'Юж федеральный округ',   # Сокращённое название федерального округа
        'спб',                    # Короткая аббревиатура
        'рт',                     # Очень короткая аббревиатура
        'город москва столица российской федерации город федерального значения',  # Полное официальное название
        'тюменская область (кроме ханты мансийского автономного округа югры и ямало ненецкого автономного округа)'  # Сложное название
    ]
})

In [10]:
result_df = matcher.match_dataframe(
    sample_data,
    'region_name',
    weights={'levenshtein': 0.4, 'token_set': 0.6},
    approach_weights={'original': 0.3, 'stemmed': 0.7},
    threshold=70
)

In [11]:
result_df

,region_name,ter,levenshtein_score
0,московск Обл,Московская область,86.30
1,свердловск,Свердловская область,82.82
2,петербург,Санкт-Петербург,90.00
3,Mосковская област,Московская область,99.10
4,татарстан респ.,Республика Татарстан,70.16
5,Свердлов обл,Свердловская область,74.30
6,aлтайский к,Алтайский край,89.76
7,Республика Алтай,Республика Алтай,100.00
8,ХМао,Ханты-Мансийский автономный округ — Югра,97.20
9,Юж федеральный округ,Южный федеральный округ,95.80


## 4. Добавление дополнительных полей из эталонного справочника

In [12]:
# Добавить английские названия
result_df = matcher.attach_field(result_df, 'region_name', 'name_eng')

# Добавить коды ОКАТО
result_df = matcher.attach_field(result_df, 'region_name', 'okato')

# Добавить коды ISO
result_df = matcher.attach_field(result_df, 'region_name', 'iso_code')

In [13]:
result_df

,region_name,ter,levenshtein_score,name_eng,okato,iso_code
0,московск Обл,Московская область,86.30,Moskovsskaya,46000000,RU-MOS
1,свердловск,Свердловская область,82.82,Sverdlovsk,65000000,RU-SVE
2,петербург,Санкт-Петербург,90.00,Sankt-Peterburg,40000000,RU-SPE
3,Mосковская област,Московская область,99.10,Moskovsskaya,46000000,RU-MOS
4,татарстан респ.,Республика Татарстан,70.16,Tatarstan,92000000,RU-TA
5,Свердлов обл,Свердловская область,74.30,Sverdlovsk,65000000,RU-SVE
6,aлтайский к,Алтайский край,89.76,Altayskiy,01000000,RU-ALT
7,Республика Алтай,Республика Алтай,100.00,Altay,84000000,RU-AL
8,ХМао,Ханты-Мансийский автономный округ — Югра,97.20,Khanty-Mansiyskiy Avtonomnyy Okrug,71100000,RU-KHM
9,Юж федеральный округ,Южный федеральный округ,95.80,Southern Federal District,00000000,None


## 5. Крайние случаи

In [14]:
edge_cases = pd.DataFrame({
    'region_name': [
        '',                    # Пустая строка
        None,                  # Значение None
        'несуществующий регион',  # Несуществующий регион
        '12345',               # Только цифры
        '   ',                 # Только пробелы
        'москва',              # Валидное, но короткое
        'МОСКВА',              # Все заглавные
        'москва!!!',           # С пунктуацией
        'москва москва',       # Дублирование слов
    ]
})

edge_results = matcher.match_dataframe(
    edge_cases,
    'region_name',
    threshold=50 
)

In [15]:
edge_results

,region_name,ter,levenshtein_score
0,,None,0.00
1,None,None,0.00
2,несуществующий регион,Республика Ингушетия,0.00
3,12345,None,0.00
4,,None,0.00
5,москва,Москва,100.00
6,МОСКВА,Москва,100.00
7,москва!!!,Москва,85.50
8,москва москва,Москва,81.25
